In [1]:
import pykep as pk
import space_navigator as snav
import numpy as np

In [2]:
EARTH_RADIUS = 6.3781e6
MU_EARTH = 398600800000000.0

In [3]:
env_path = 'complex.env'

In [4]:
# a (semi-major axis): meters
altitude = np.array([
    686.0118,
    685.4915,
    685.4774,
    686.1085,
    686.0851,
])
a = altitude + EARTH_RADIUS # TODO: уточнить

# e (eccentricity): 0 < e < 1
e = np.array([
    0.000602,
    0.000726,
    0.000711,
    0.000730,
    0.000728,
])

# i (inclination): radians
inc = np.radians([
    98.288,
    94.123,
    94.325,
    97.394,
    98.118,
])

# W (Longitude of the ascending node): radians
W = np.radians([
    334.084,
    353.603,
    353.441,
    346.278,
    345.912,
])

# w (Argument of periapsis): radians
w = np.radians([
    329.626,
    336.080,
    337.108,
    335.771,
    335.712,
])

# M (mean anomaly): radians
M = np.radians([
    276.353,
    271.929,
    270.931,
    272.011,
    271.991,
])

In [5]:
names = [
    'K2',
    'Debris1',
    'Debris2',
    'Debris3',
    'Debris4',
]

In [6]:
# Maneuver start time (1.5 orbit before first collision)
start_epoch = pk.epoch_from_string("2011-03-30 02:58:26.270")
collision_epochs = {
    "Debris1": pk.epoch_from_string("2011-03-30 10:50:29.255"),
    "Debris2": pk.epoch_from_string("2011-03-30 07:33:09.015"),
    "Debris3": pk.epoch_from_string("2011-03-30 13:16:18.125"),
    "Debris4": pk.epoch_from_string("2011-03-30 05:02:23.052"),
}

In [43]:
# попробовать epoch = 0 в jd или еще где
epoch = collision_epochs["Debris4"].mjd2000
start_time = start_epoch.mjd2000
end_time = collision_epochs["Debris3"].mjd2000 + 0.001

In [75]:
# epoch = 0
# start_time =  collision_epochs["Debris4"].mjd2000
# end_time = start_time + 0.3

# epoch = collision_epochs["Debris4"].mjd2000
# start_time = epoch
# end_time = start_time + 0.3

# epoch = collision_epochs["Debris4"].mjd2000
# start_time = epoch
# end_time = start_time + 3

In [76]:
start_epoch

2011-Mar-30 02:58:26.270000

In [77]:
mu_protected = 0.1 #?
radius_protected = 20 #?
safe_radius_protected = 20 #?
fuel = 10

mu_debris = 0.1 #?
radius_debris = 0.1
safe_radius_debris = 0.1

In [78]:
collision_epochs

{'Debris1': 2011-Mar-30 10:50:29.255000,
 'Debris2': 2011-Mar-30 07:33:09.015000,
 'Debris3': 2011-Mar-30 13:16:18.125000,
 'Debris4': 2011-Mar-30 05:02:23.052000}

In [79]:
protected_str = f"""{names[0]}
{epoch}
{a[0]}, {e[0]}, {inc[0]}, {W[0]}, {w[0]}, {M[0]}
{MU_EARTH}, {mu_protected}, {radius_protected}, {safe_radius_protected}
{fuel}
"""

debris_str = ""
for i in range(1, 5):
    debris_str += f"""{names[i]}
{epoch}
{a[i]}, {e[i]}, {inc[i]}, {W[i]}, {w[i]}, {M[i]}
{MU_EARTH}, {mu_debris}, {radius_debris}, {safe_radius_debris}
0
"""

env_str = f"""{start_time}, {end_time}
osc
""" + protected_str + debris_str

In [80]:
# protected_str = f"""{names[0]}
# {start_time}
# {a[0]}, {e[0]}, {inc[0]}, {W[0]}, {w[0]}, {M[0]}
# {MU_EARTH}, {mu_protected}, {radius_protected}, {safe_radius_protected}
# {fuel}
# """

# debris_str = ""
# for i in range(1, 5):
#     debris_str += f"""{names[i]}
# {collision_epochs[names[i]].mjd2000}
# {a[i]}, {e[i]}, {inc[i]}, {W[i]}, {w[i]}, {M[i]}
# {MU_EARTH}, {mu_debris}, {radius_debris}, {safe_radius_debris}
# 0
# """

# env_str = f"""{start_time}, {end_time}
# osc
# """ + protected_str + debris_str

In [81]:
with open(env_path, 'w') as f:
    f.write(env_str)

In [82]:
env = snav.utils.read_environment(env_path)

In [83]:
agent = snav.agent.TableAgent()
# simulator = snav.simulator.Simulator(agent, env)

In [84]:
data = snav.models.collision_data(env, 1e-6, agent)
for coll in data:
    print(coll['debris_name'], coll['epoch'])

In [54]:
# проверка (что вообще учитывается + что расстояния похожие)
# как привязать сюда дату?